![CC](https://i.creativecommons.org/l/by/4.0/88x31.png)

This work is licensed under a [Creative Commons Attribution 4.0 International License](http://creativecommons.org/licenses/by/4.0/).

# Controlling the flow past a cylinder with OpenFOAM and PyTorch

**Note: this exercise still needs to be updated for the winter term 2022/2023.**

This exercise consists of two parts, which are meant to be solved in two individual sessions. We will implement two active flow control approaches to reduce drag and lift forces acting on the surface of a circular cylinder in a stream. The base simulation is the same as in exercises 2 and 9. In both control approaches, the flow is actuated by rotating the cylinder. The rotation is easily implemented in a CFD simulation by modifying the velocity boundary condition applied to the *cylinder* patch. Say that $u_\parallel$ is the signed cross-radial speed of the cylinder's surface. Then the surface velocity vector field $\mathbf{u}_\parallel$ is obtained by multiplying the speed with tangential unit vector $\mathbf{t}_\parallel (\mathbf{x})$:
$$
  \mathbf{u}_\parallel = u_\parallel \mathbf{t}_\parallel (\mathbf{x}),
$$
where $\mathbf{t}_\parallel (\mathbf{x})$ is tangential to the cylinder's surface (note that the simulation is 2D, so there is only one sensible tangent). The two control approaches vary in the way how they determine $u_\parallel$. In open-loop control, the control law is a function of time, e.g., $u_\parallel = u_\parallel (t)$. The closed-loop control law is a function of the pressure sensors $\mathbf{p}$, e.g., $u_\parallel = u_\parallel (\mathbf{p})$.

## Open-loop active flow control

As observed in exercise 2, the uncontrolled flow past the cylinder at $Re=100$ yields almost perfectly sinusoidal lift and drag curves in the quasi-steady state (neglecting the non-zero mean). Therefore, we could try to interrupt the natural vortex shedding by rotating the cylinder periodically in alternating directions. A suitable control law might be:
$$
  u_\perp (t) = A \mathrm{sin}(2\pi f t),
$$
where the amplitude $A$ and the frequency $f$ are adjustable parameters. To implement this new velocity boundary condition, we use OpenFOAM's [inline expressions for dictionaries](https://www.openfoam.com/news/main-news/openfoam-v1912/pre-processing#pre-processing-expressions-syntax). To get started, create a dedicated folder and make a copy of the base simulation:
```
mkdir exercises/openloop_control
cp -r test_cases/cylinder2D/ exercises/openloop_control/A2f6
```
The velocity boundary condition is specified in the *0.org/U* dictionary. Open the dictionary and make the following modifications:

1. currently, there is one entry for the *cylinder*, *top*, and *bottom* patches, because the same boundary condition is applied to these patches; remove the *cylinder* patch from the joint entry and create a new separate entry only for *cylinder*
2. use the *exprFixedValue* type boundary condition as follows:
```
cylinder
{
  type        exprFixedValue;
  value       $internalField;
  amplitude   2.0;
  frequency   6.0;
  valueExpr   "(vector(0, 0, 1) ^ face()) / area() * ???";
}
```
Variables can be accessed using the syntax `$variable`. E.g., `$frequency` becomes 6.0 in the example above. The expression `(vector(0, 0, 1) ^ face()) / area()` computes the tangential unit vector. The only missing piece is the expression for $u_\perp (t)$ in place of the three question marks. You can find the names for the required mathematical functions in the [documentation](https://www.openfoam.com/documentation/guides/latest/doc/openfoam-guide-expression-syntax.html).

One additional modification of the workflow is necessary when running the simulation with the Singularity container. The default workflow expects the Singularity image to be stored in the simulation's parent folder, but for this exercise, we created one more subfolder, namely *openloop_control*. To modify the default behavior, open the *Allrun.singularity* script and change/add the following lines:
```
# replace . ../../functions with
. ../../../functions
# and add the line
setImage "../../../of2106.sif"
# before using the image for the first time
```
Once the execution is complete, open the case in ParaView:
```
paraview post.foam
```
and apply the following steps to check if the control works:
1. set *Case Type* to *Decomposed Case*
2. under *Mesh Regions*, select only the cylinder patch
3. click *Apply*
4. create a *Glyph* filter and set the *Scale Array* to *U*
5. click the play button and observe the surface velocity field

If the control is implemented correctly, we can now check its effect on the forces acting on the cylinder. To do so, create a new Jupyter notebook, load the drag and lift coefficients as in previous exercises, and plot the controlled against the un-controlled coefficients. 

Finally, create one or two additional parameter variations in terms of $f$ and $A$, and repeat the process of evaluating the force coefficients.

## Closed-loop active flow control

The agent's aim in the DRL-based closed-loop control is to learn a control law $u_\parallel = u_\parallel (\mathbf{p})$ by trail and error. First, copy the initial setup to the exercise folder:
```
cp -r test_cases/drl_control_cylinder exercises/
```
The controller is implemented as an OpenFOAM boundary condition (BC). In the BC implementation, actions (angular velocities) are sampled and applied based on the policy's output. To compile the BC, go to the repository's top level, log into the Singularity container, and execute the following commands:
```
singularity shell of2106.sif
Singularity> source /lib/openfoam/openfoam2106/etc/bashrc
Singularity> cd exercises/drl_control_cylinder/agentRotatingWallVelocity/
wmake
# check if the library file libAgentRotatingWallVelocity.so was created
# and than leave the container
ls
exit
```
Now we are ready to start the training. Navigate to the *drl_control_cylinder* folder and initiate the training by running the *main.py* script:
```
cd exercises/drl_control_cylinder/
python3 main.py
```
The output created during the training is mainly stored in two directories:
- *Data*: contains the trajectory and force coefficient data for each PPO iteration
- *results*: contains three subfolders:
  - *evaluation*: Numpy binary data holding the mean reward after each PPO iteration, e.g, *evaluations_0.npy* contains the mean reward after iteration *0*
  - *models*: snapshots of the policy model after each PPO iteration
  - *value_models*: snapshots of the value model after each PPO iteration

Meanwhile the training is running, write three scripts that:
1. visualize the mean rewards as the learning progresses
2. visualize the mean angular velocity as the training progresses
3. visualize the drag and lift curves as the training progresses

The first two array entries of each *evaluations_i.npy* file hold mean and standard deviation of all rewards obtained during the *i*th PPO iteration. The predicted mean angular velocity of each control step and simulation is stored in the corresponding *trajectory.csv* file. The drag and lift coefficients are also available in the *Data* folder. Use the *read_data_from_trajectory* function to load trajectory and coefficient data from the disk.

Once the training is completed (approximately after 1h), test the best policy as follows:
1. copy the policy associated with the highest mean reward from the *results* folder to *env/base_case/test_policy/policy.pt*
2. navigate to the *test_policy* folder; in the file *0.org/U*, set *train* to *false* and *absOmegaMax* to *10.0*
3. run the simulation by executing the *Allrun.singularity* script
4. visualize drag and lift curves once the simulation is completed
5. animate the controlled flow using ParaView

**Congratulations! This completes the final two exercise session.**